In [1]:
import pandas as pd
import os
from tqdm.notebook import tqdm
import numpy as np
import json
from datetime import datetime

# 경고 무시 모듈
import warnings

# 경고를 무시
warnings.filterwarnings(action='ignore')

# 경고 보이기  
#warnings.filterwarnings(action='default')

In [2]:
os.listdir('../전처리/wi-fi/path/2021/')

['pre_2021.csv', '개소별']

In [21]:
df_2022 = pd.read_csv('../전처리/wi-fi/path/presence_path_2022.csv', encoding='cp949')

In [ ]:
df_2021 = pd.read_csv('../전처리/wi-fi/path/presence_path_2021.csv', encoding='cp949')

In [2]:
# 용량 줄이기
# 데이터 범위 체크 후, 최적화된 타입 적용
def reduce_mem_usage(df):
    
    # 현재 메모리 사용량
    start_mem = df.memory_usage().sum() / 1024**2
    print('데이터 프레임의 메모리 사용량 : {:.2f} MB'.format(start_mem))
    
    
    # 컬럼 하나씩 확인 후 적용
    for col in tqdm(df.columns[1:]):
        col_type = df[col].dtype

        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
        else:
            df[col] = df[col].astype('category')
            
    # 타입변경 후 메모리 사용량        
    end_mem = df.memory_usage().sum() / 1024**2
    print('최적화 후 메모리 사용량 : {:.2f} MB'.format(end_mem))
    print('감소율 : {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))
    
    return df

In [3]:
# 데이터프레임 타입변경
def df_astype(df):
    reduce_mem_usage(df)
    df['STARTTIMESTAMP'] = pd.to_datetime(df['STARTTIMESTAMP'])
    df['ENDTIMESTAMP'] = pd.to_datetime(df['ENDTIMESTAMP'])

In [ ]:
df_astype(df_2022)

In [ ]:
df_astype(df_2021)

In [ ]:
# 동일한 위경도값을 보유한 포인트를 1개의 개소로 측정

# 위경도 csv 파일 읽기
wk = pd.read_csv('../전처리/wi-fi/테마형특화단지ap_위경도.csv', encoding='cp949', index_col=['MAC주소'])


# MAC 코드화 딕셔너리 불러오기
with open('../mac_code.json', 'r') as f :
    mac_code = json.load(f, encoding='utf-8')

# key값과 values값 위치 변경
mac_code = dict(zip(mac_code.values(),mac_code.keys()))

In [ ]:
# 사용되지 않은 Wi-Fi, 목록에서 제거
for n in wk.index:
    if n not in mac_code.values():
        wk.drop(n, axis=0, inplace=True)

# 위도로 정렬 (같은 위도 묶기)
wk.sort_values('위도', inplace=True)

In [ ]:
# 이전 Mac 주소를 담을 변수
f_mac = None

# 동일한 위경도를 보유한 Mac 주소 리스트
repeat = []

# repeat 리스트를 모은 리스트
repeat_all = []

for n in wk.index:
    # 첫행 Skip
    if n == wk.iloc[0].name:
        f_mac = n
        continue
    # 이전 행 위도와 같다면 append
    if wk.loc[f_mac]['위도'] == wk.loc[n]['위도']:
        repeat.append(f_mac)
        repeat.append(n)
    
    # 전체 리스트 
    else:
        # 빈 리스트는 무시
        if len(repeat) == 0:
            f_mac = n
            continue
        repeat_all.append(list(set(repeat)))
        repeat = []
    f_mac = n

In [ ]:
with open('../mac_code.json', 'r') as f :
    mac_code = json.load(f, encoding='utf-8')

repeat_num_all = []
repeat_num = []
for n in repeat_all:
    for i in range(len(n)):
        repeat_num.append(mac_code[n[i]])
    repeat_num_all.append(repeat_num)
    repeat_num = []
    
    
# key값과 values값 위치 변경
mac_code = dict(zip(mac_code.values(),mac_code.keys()))


for n in repeat_num_all:
    for i in range(len(n)):
        if i == 0:
            continue
        mac_code[n[i]] = mac_code[n[0]]

In [ ]:
def ch_mac_code(df): 
    for x in tqdm(range(len(df))):
        df['MAC_add'][x] = mac_code[df['MAC_add'][x]]
        if np.isnan(df['Next_MAC'][x]):
            continue
        else:
            df['Next_MAC'][x] = mac_code[df['Next_MAC'][x]]

In [ ]:
ch_mac_code(df_2022)

In [ ]:
ch_mac_code(df_2021)

In [ ]:
df_2022.to_csv('./wi-fi_개소_통합/2022년.csv', encoding='cp949', index=False)
df_2021.to_csv('./wi-fi_개소_통합/2021년.csv', encoding='cp949', index=False)

In [4]:
df_2021 = pd.read_csv('./wi-fi_개소_통합/2021년.csv', encoding='cp949')

In [ ]:
df_2022 = pd.read_csv('./wi-fi_개소_통합/2022년.csv', encoding='cp949')

In [5]:
df_astype(df_2021)

데이터 프레임의 메모리 사용량 : 2705.92 MB


  0%|          | 0/7 [00:00<?, ?it/s]

최적화 후 메모리 사용량 : 1604.07 MB
감소율 : 40.7%


In [ ]:
df_astype(df_2022)

In [15]:
# 월별 원본파일
def month_split(DF, year, encoding='cp949'):
    # 년도별 파일의 최대 월 찾기
    length = DF['STARTTIMESTAMP'].dt.month.max()

    # 년도 폴더 생성
    try:
        directory_01 = './전처리/split_원본/' + f'{year}년_월별'
        if not os.path.exists(directory_01):
            os.makedirs(directory_01)
    except OSError:
        print ('Error: Creating directory. ' +  directory_01)
    
    # 월별 나누기
    print('전체 진행도------------------------------------------------------')
    for n in tqdm(range(1, length+1)):
        if n <= 9:
            num = '0' + str(n)
        else:
            num = str(n)

        # 기간설정(월별)
        if n != 12:
            df = DF[(DF['STARTTIMESTAMP'] >= datetime.strptime(f'{year}-{n}-01 00:00:00',
                                                              '%Y-%m-%d %H:%M:%S')) &
                    (DF['STARTTIMESTAMP'] < datetime.strptime(f'{year}-{n+1}-01 00:00:00',
                                                              '%Y-%m-%d %H:%M:%S'))]
        else:
            df = DF[DF['STARTTIMESTAMP'] >= datetime.strptime(f'{year}-{n}-01 00:00:00', 
                                                              '%Y-%m-%d %H:%M:%S')]
        
        

        # 월별 csv 파일 저장    
        df.to_csv(f'{directory_01}/pre_{year}_{num}월.csv', encoding=encoding, index=False)

In [16]:
month_split(df_2021, 2021)

전체 진행도------------------------------------------------------


  0%|          | 0/12 [00:00<?, ?it/s]

In [ ]:
month_split(df_2022, 2022)

In [11]:
def buffer_250_month(month, year):
    
    
    df = pd.read_csv(f'./전처리/와이파이_개소별_사용자/csv/{year}년/{year}년_{month}월.csv', encoding='cp949', index_col=['MAC_add'])
    
    # Wi-Fi 개소별 사용량 count
    df_month = df[['User_count']]
    
    # 테마구역별 Wi-Fi MAC주소
    먹자골목 = ['7C573CCF4CCC', '7C573CCF456E','7C573CCF457A','7C573CCF465E','7C573CCF454E','7C573CCF497E']
    신세계백화점 = ['7C573CCF43EE','7C573CCF4F78','7C573CCF4BD4']
    수로왕릉 = ['7C573CCF471C','7C573CCF4B98','D015A6C7F506','D015A6C7F4F2','7C573CCF4AAE','7C573CCF4B7C']
    대성동고분박물관 = ['7C573CCF4B2E','7C573CCF49F8','7C573CCF4AE0','7C573CCF4AD6','7C573CCF484E','7C573CCF4B80','7C573CCF4E28','7C573CCF4CF0','7C573CCF4F1E']
    김해민속박물관 = ['7C573CCF4D54','7C573CCF4A94','7C573CCF4302','7C573CCF4F62','7C573CCF4EA6']
    김해박물관 = ['7C573CCF4A6E','7C573CCF4BDA','7C573CCF4CE8','7C573CCF49AA']
    수로왕비릉 = ['7C573CCF4F88','7C573CCF4A90']
    김해시민의종 = ['7C573CCF4D0C','7C573CCF4E8C','7C573CCF4320']
    연지공원 = ['7C573CCF4A7E','7C573CCF4D6A','7C573CCF45F4','7C573CCF4526','7C573CCF44C0','7C573CCF43DE','7C573CCF49C0','7C573CCF4C3E','7C573CCF4AEE','7C573CCF4740','7C573CCF455A']
    거북공원 = ['7C573CCF42BE','7C573CCF47E6','7C573CCF44DA']
    봉황대공원 = ['7C573CCF4AF2','7C573CCF4AFE','7C573CCF4EF2','7C573CCF47D0','7C573CCF4D94','7C573CCF4C04','7C573CCF4B36']
    봉황역 = ['7C573CCF4E54','7C573CCF4AE6','7C573CCF48D4','7C573CCFB76A']
    수로왕릉역 = ['7C573CCF45BA']
    박물관역 = ['7C573CCF46C8']

    wifi_all = [먹자골목, 신세계백화점 , 수로왕릉, 대성동고분박물관, 김해민속박물관, 
                김해박물관, 수로왕비릉, 김해시민의종, 연지공원, 거북공원, 봉황대공원, 봉황역, 수로왕릉역, 박물관역]
    wifi_name = ['먹자골목', '신세계백화점' , '수로왕릉', '대성동고분박물관', '김해민속박물관', 
                '김해박물관', '수로왕비릉', '김해시민의종', '연지공원', '거북공원', '봉황대공원', '봉황역', '수로왕릉역', '박물관역']
    
    for mac_l, name in tqdm(zip(wifi_all, wifi_name)):
        a = []

        for n in mac_l:
            if n in df_month.index:
                a.append(n)

        # 폴더 생성
        try:
            directory = f'./전처리/250m_반경_구역/{year}년/{month}월'
            if not os.path.exists(directory):
                os.makedirs(directory)
        except OSError:
            print ('Error: Creating directory. ' +  directory)
        
        
        # 테마 구역별 Wi-Fi 사용량 합산
        df = pd.DataFrame(df_month.loc[a].sum(), columns=['Wi-Fi_user'])

        df.to_csv(f'{directory}/{name}.csv',encoding='cp949', index=False)

In [12]:
# 2021년
for n in tqdm(range(1, 13)):
    buffer_250_month(n, 2021)

  0%|          | 0/12 [00:00<?, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

...

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

In [13]:
# 2022년
for n in tqdm(range(3, 11)):
    buffer_250_month(n, 2022)

  0%|          | 0/8 [00:00<?, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

In [29]:
year = 2021
for month in tqdm(range(1, 12 + 1)):

    df = pd.read_csv(f'./전처리/와이파이_개소별_사용자/csv/{year}년/{year}년_{month}월.csv', encoding='cp949', index_col=['MAC_add'])

    # Wi-Fi 개소별 사용량 count
    df_month = df[['User_count']]
    df_month.reset_index(inplace=True)

    if month == 1:
        merge_df = df_month
        continue

    merge_df = pd.merge(merge_df, df_month, how='outer',on='MAC_add')


  0%|          | 0/12 [00:00<?, ?it/s]

In [30]:
merge_df

MAC_add  User_count_x  User_count_y  User_count_x  User_count_y  \
0   7C573CCF4B36        261671        226013         80928         79556   
1   7C573CCF4CE8         72957         65298         30497         27830   
2   7C573CCF4AE6        107771        101106         53049         46726   
3   7C573CCF45BA         73058         73173         26623         30250   
4   7C573CCF465E        181237        138608         60343         52173   
5   7C573CCF46C8        104599         90704         40243         43982   
6   7C573CCF4518        256904        200496         71169         62734   
7   7C573CCF454E        159470        133509         53411         48937   
8   7C573CCF4AEE        115848        104899         38505         29387   
9   7C573CCF4D74         50730         43356         12542         10677   
10  7C573CCF470A        113725         97683         41397         33300   
11  7C573CCFB76A        130183        106479         45294         42859   
12  7C573CCF42BE        125978        111950         35313         30353   
13  7C573CCF47E6         72128         65375         18584         16211   
14  7C573CCF4E8C         49388         43854         18949         18593   
15  7C573CCF4F78        212819        177637         48691         42409   
16  7C573CCF43DE         27394         27183         13608          8550   
17  7C573CCF44DA         98700         92083         25790         24935   
18  7C573CCF4398        166263        136664         43033         37423   
19  7C573CCF43EE        106332         85276         34279         29041   
20  7C573CCF4AD6         17270         18092          5173          5838   
21  7C573CCF4320        110287         89073         35157         30972   
22  7C573CCF456E        169291        133342         43992         37769   
23  7C573CCF48D4        132817        110696         43540         40069   
24  7C573CCF4CCC        131014        113224         29618         24793   
25  7C573CCF497E        163978        135579         44138         38868   
26  7C573CCF4ECC         52596         54984         21111         19933   
27  7C573CCF457A        133154        110691         32393         26810   
28  7C573CCF49C0         32101         34664         17889         12176   
29  7C573CCF4740         76003         66681         24051         19346   
30  7C573CCF4526         29918         25746         12606          9373   
31  7C573CCF4F88         43923         37210         16451         15379   
32  7C573CCF4B7C         10769         13305          5001          5328   
33  7C573CCF44C0         25557         23355         12011          7252   
34  7C573CCF4EF2         57946         52378         22812         19229   
35  7C573CCF471C         25919         22617         10676         11527   
36  7C573CCF4D6A         46592         42563         22899         15368   
37  7C573CCF455A         37645         38276         16499         11956   
38  7C573CCF4B80         48779         49645         19374         19151   
39  7C573CCF4342         51356         49993         11296          8753   
40  7C573CCF4EA6         44836         49516         18918         16074   
41  7C573CCF45F4         40397         36261         19280         13777   
42  7C573CCF47D0         81060         72905         26018         25562   
43  7C573CCF49F8         24864         23230          7085          7044   
44  D015A6C7F506          8588          9464          2993          2859   
45  7C573CCF4A94         22169         28803          8124          9614   
46  7C573CCF4D0C         35353         36868         17973         15770   
47  7C573CCF4B2E         17526         19015          7553          7629   
48  7C573CCF49AA         50293         48092         20407         15643   
49  7C573CCF4AAE          7962          9626          3973          4312   
50  7C573CCF484E         26842         24469          9976         12316   
51  7C573CCF4B98         24253         24083         1220

In [31]:
 merge_df['User_count'] = merge_df.sum(axis=1)

In [33]:
def buffer_250_year(year, s_month, e_month):

    merge_df = None

    for month in tqdm(range(s_month, e_month + 1)):
        
        
        
        df = pd.read_csv(f'./전처리/와이파이_개소별_사용자/csv/{year}년/{year}년_{month}월.csv', encoding='cp949', index_col=['MAC_add'])
    
        # Wi-Fi 개소별 사용량 count
        df_month = df[['User_count']]
        df_month.reset_index(inplace=True)

        if month == s_month:
            merge_df = df_month
            continue

        merge_df = pd.merge(merge_df, df_month, how='outer',on='MAC_add')



    merge_df['User_count'] = merge_df.sum(axis=1)
    merge_df.drop(['User_count_x','User_count_y'], axis=1, inplace=True)
    merge_df = merge_df.astype({'User_count':int})
    merge_df.set_index('MAC_add',inplace=True)


    # 테마구역별 Wi-Fi MAC주소
    먹자골목 = ['7C573CCF4CCC', '7C573CCF456E','7C573CCF457A','7C573CCF465E','7C573CCF454E','7C573CCF497E']
    신세계백화점 = ['7C573CCF43EE','7C573CCF4F78','7C573CCF4BD4']
    수로왕릉 = ['7C573CCF471C','7C573CCF4B98','D015A6C7F506','D015A6C7F4F2','7C573CCF4AAE','7C573CCF4B7C']
    대성동고분박물관 = ['7C573CCF4B2E','7C573CCF49F8','7C573CCF4AE0','7C573CCF4AD6','7C573CCF484E','7C573CCF4B80','7C573CCF4E28','7C573CCF4CF0','7C573CCF4F1E']
    김해민속박물관 = ['7C573CCF4D54','7C573CCF4A94','7C573CCF4302','7C573CCF4F62','7C573CCF4EA6']
    김해박물관 = ['7C573CCF4A6E','7C573CCF4BDA','7C573CCF4CE8','7C573CCF49AA']
    수로왕비릉 = ['7C573CCF4F88','7C573CCF4A90']
    김해시민의종 = ['7C573CCF4D0C','7C573CCF4E8C','7C573CCF4320']
    연지공원 = ['7C573CCF4A7E','7C573CCF4D6A','7C573CCF45F4','7C573CCF4526','7C573CCF44C0','7C573CCF43DE','7C573CCF49C0','7C573CCF4C3E','7C573CCF4AEE','7C573CCF4740','7C573CCF455A']
    거북공원 = ['7C573CCF42BE','7C573CCF47E6','7C573CCF44DA']
    봉황대공원 = ['7C573CCF4AF2','7C573CCF4AFE','7C573CCF4EF2','7C573CCF47D0','7C573CCF4D94','7C573CCF4C04','7C573CCF4B36']
    봉황역 = ['7C573CCF4E54','7C573CCF4AE6','7C573CCF48D4','7C573CCFB76A']
    수로왕릉역 = ['7C573CCF45BA']
    박물관역 = ['7C573CCF46C8']

    wifi_all = [먹자골목, 신세계백화점 , 수로왕릉, 대성동고분박물관, 김해민속박물관, 
                김해박물관, 수로왕비릉, 김해시민의종, 연지공원, 거북공원, 봉황대공원, 봉황역, 수로왕릉역, 박물관역]
    wifi_name = ['먹자골목', '신세계백화점' , '수로왕릉', '대성동고분박물관', '김해민속박물관', 
                '김해박물관', '수로왕비릉', '김해시민의종', '연지공원', '거북공원', '봉황대공원', '봉황역', '수로왕릉역', '박물관역']

    for mac_l, name in tqdm(zip(wifi_all, wifi_name)):
        a = []

        for n in mac_l:
            if n in merge_df.index:
                a.append(n)

        # 폴더 생성
        try:
            directory = f'./전처리/250m_반경_구역/{year}년/{year}년'
            if not os.path.exists(directory):
                os.makedirs(directory)
        except OSError:
            print ('Error: Creating directory. ' +  directory)


        # 테마 구역별 Wi-Fi 사용량 합산
        df = pd.DataFrame(merge_df.loc[a].sum(), columns=['Wi-Fi_user'])

        df.to_csv(f'{directory}/{name}.csv',encoding='cp949', index=False)

In [34]:
buffer_250_year(2022, 3, 10)

  0%|          | 0/8 [00:00<?, ?it/s]

0it [00:00, ?it/s]

In [35]:
buffer_250_year(2021, 1, 12)

  0%|          | 0/12 [00:00<?, ?it/s]

0it [00:00, ?it/s]